In [1]:
transacoes = [
    ['Banana', 'Leite'],
    ['Banana', 'Laranja', 'Leite', 'Ovos'],
    ['Banana', 'Laranja', 'Leite'],
    ['Laranja', 'Ovos'],
    ['Banana', 'Laranja', 'Ovos'],
    ['Banana', 'Laranja', 'Leite', 'Ovos'],
    ['Laranja', 'Leite'],
    ['Banana', 'Laranja', 'Leite']
]

### FP-Tree

**A FP-Tree organiza os itens frequentes em caminhos compartilhados**. Se várias transações têm os mesmos itens, eles são armazenados uma única vez na árvore.
- Em vez de guardar todas as transações repetidamente, a FP-Tree armazena apenas o necessário para encontrar os padrões frequentes.
- Isso reduz o uso de memória e acelera o processo de mineração.

In [ ]:
class FPNode:
    def __init__(self, item, count, parent):
        self.item = item  # Nome do item
        self.count = count  # Contagem do item
        self.parent = parent  # Nó pai
        self.children = {}  # Nós filhos
        self.next = None  # Ponteiro para o próximo nó com o mesmo item (lista encadeada)

    def increment(self, count):
        self.count += count

# Aqui podemos definir o suporte minimo como a frequencia minima
def construir_fp_tree(transacoes, suporte_minimo):
    
    contagem_itens = {}
    for transacao in transacoes:
        for item in transacao:
            contagem_itens[item] = contagem_itens.get(item, 0) + 1 # se não existe chave, coloca 0 + 1 para iniciar chave

    # Remover itens que não atendem ao suporte mínimo
    itens_frequentes = { item: count for item, count in contagem_itens.items() if count >= suporte_minimo }
    if not itens_frequentes:
        return None, None

    # Ordenar os itens por frequência (decrescente)
    ordem_itens = sorted(itens_frequentes.keys(), key=lambda x: itens_frequentes[x], reverse=True)

    # Construir o FP-Tree
    raiz = FPNode(None, None, None)
    tabela_encadeamento = {}

    for transacao in transacoes:
        
        # Ordena os itens da transação de acordo com a ordem global definida anteriormente
        ordered_item_set = [item for item in transacao if item in itens_frequentes]
        ordered_item_set.sort(key=lambda x: ordem_itens.index(x))

        no_atual = raiz
        for item in ordered_item_set: # Para cada item na transação ordenada pela frequencia...
            
            if item in no_atual.children: # Se o item já está no nó atual
                no_atual.children[item].increment(1) # Incrementa ele no nó atual
            
            else: 
                # Caso o item não esteja cria um novo nó com ele e contagem atual = 1
                novo_no = FPNode(item, 1, no_atual)
                no_atual.children[item] = novo_no

                # Atualiza a tabela de encadeamento
                if item in tabela_encadeamento:
                    no_anterior = tabela_encadeamento[item]
                    while no_anterior.next is not None:
                        no_anterior = no_anterior.next
                    no_anterior.next = novo_no
                else:
                    tabela_encadeamento[item] = novo_no

            # Atualiza o nó atual como sendo o nó que a gente acabou de incrementar
            # Na próxima iteração, vamos ver se o próximo item da lista transação ordenada está
            # nos filhos do nó que a gente acabou de incrementar.
            no_atual = no_atual.children[item]

    return raiz, tabela_encadeamento




{'Banana': <__main__.FPNode object at 0x000001D11F57D220>, 'Leite': <__main__.FPNode object at 0x000001D11F57D2B0>, 'Laranja': <__main__.FPNode object at 0x000001D11F57D250>, 'Ovos': <__main__.FPNode object at 0x000001D11F57D490>}


In [7]:
def print_fp_tree(node, indent=0):
    if node is None:
        return

    print("  " * indent + f"Item: {node.item}, Count: {node.count}")

    for child_item, child_node in node.children.items():
        print_fp_tree(child_node, indent + 1)

def print_tabela_encadeamento(tabela_encadeamento):
    print("\nTabela de Encadeamento:")
    for item, node in tabela_encadeamento.items():
        print(f"Item: {item}")
        no_atual = node
        while no_atual is not None:
            print(f"  Node: {no_atual.item}, Count: {no_atual.count}")
            no_atual = no_atual.next

fp_tree, tabela_encadeamento = construir_fp_tree(transacoes, 2)
print("FP-Tree:")
print_fp_tree(fp_tree)
print_tabela_encadeamento(tabela_encadeamento)

FP-Tree:
Item: None, Count: None
  Item: Banana, Count: 1
    Item: Leite, Count: 1
  Item: Laranja, Count: 7
    Item: Banana, Count: 5
      Item: Leite, Count: 4
        Item: Ovos, Count: 2
      Item: Ovos, Count: 1
    Item: Ovos, Count: 1
    Item: Leite, Count: 1

Tabela de Encadeamento:
Item: Banana
  Node: Banana, Count: 1
  Node: Banana, Count: 5
Item: Leite
  Node: Leite, Count: 1
  Node: Leite, Count: 4
  Node: Leite, Count: 1
Item: Laranja
  Node: Laranja, Count: 7
Item: Ovos
  Node: Ovos, Count: 2
  Node: Ovos, Count: 1
  Node: Ovos, Count: 1


### Vantagens da FP-Tree

O apriori precisa varrer o dataset várias vezes para calcular o suporte dos candidatos

O FP-Growth varre o dataset **duas vezes apenas**:
1. Conta a frequência dos itens individuais
2. Constrói a FP-Tree

Com a construção da FP-Tree, a mineração é feita diretamente na árvore, sem a necessidade de acessar o dataset novamente.